In [ ]:
import xlsxwriter 
xlsxwriter.utility.xl_col_to_name(10)

In [ ]:
import tkinter as tk
window = tk.Tk()
button = tk.Button(
    text="Click me!",
    width=25,
    height=5,
    bg="blue",
    fg="yellow",
)
greeting = tk.Label(text="Hello, Tkinter")
greeting.pack()
button.pack()

window.mainloop()

In [ ]:

from tkinter import Tk, Label
root = Tk()
w = Label(root, text="Hello, world!")
w.pack()
root.mainloop()

In [ ]:
# # Check if columns' list and labels matches -
# # this script would stop otherwise
# if csv_df.columns.all() == google_df.columns.all():
    

#     # Check if our CSV has new rows over GSheets
#     if difference > 0:
#         print("They differ by ", difference, ' strings!')

#     # If yes, we push new lines to GSheets in a cycle
#         current_row = google_len + 1

#         for i in range(difference+1):
#             taken_row = csv_df.iloc[current_row-2].astype('str')
            
#             # constructing an Excel-style pointer
#             diff_range = str('A' + str(current_row) + ':S' + str(current_row))
            
#             # it double checks if'd write into an empty row
#             if google_sheet.sheet1.cell(current_row,1).value == None:
#                 google_sheet.sheet1.update(values =
#                                         [taken_row.values.tolist()],
#                                         range_name = diff_range)
#             current_row+=1
#         print('Tables are in sync.')
    
#     # If it's the oposite or they are equal,
#     # nothing would happen   
#     elif difference < 0:
#         print('Your CSV has less entries than Google Sheets. Wrong SCV?')
#     else: print('Tables are in sync. No actions needed.')

# else: print ('Column labels don\'t match. Wrong tables?')